## Optimizing `max_mse` parameter of `MeanSquaredErrorSceneDetector`

In [1]:
from video699.configuration import get_configuration
from video699.video.annotated import (
    AnnotatedSampledVideoScreenDetector,
    AnnotatedSampledVideoPageDetector,
    evaluate_event_detector,
    get_videos,
)
from video699.video.scene import MeanSquaredErrorSceneDetector
from video699.quadrangle.rtree import RTreeDequeConvexQuadrangleTracker
from video699.event.screen import ScreenEventDetector


CONFIGURATION = get_configuration()['MeanSquaredErrorSceneDetector']

In [2]:
def accuracy(max_mse):
    CONFIGURATION['max_mse'] = str(max_mse)
    num_successes_total = 0
    num_trials_total = 0
    for annotated_video in get_videos().values():        
        convex_quadrangle_tracker = RTreeDequeConvexQuadrangleTracker(2)
        screen_detector = AnnotatedSampledVideoScreenDetector()
        page_detector = AnnotatedSampledVideoPageDetector()
        video = MeanSquaredErrorSceneDetector(annotated_video)
        screen_event_detector = ScreenEventDetector(
            video,
            convex_quadrangle_tracker,
            screen_detector,
            page_detector,
        )
        num_successes, num_trials = evaluate_event_detector(annotated_video, screen_event_detector)
        num_successes_total += num_successes
        num_trials_total += num_trials
    accuracy = 100.0 * num_successes_total / num_trials_total
    return accuracy

In [3]:
def binary_search(min_accuracy):
    lower_limit = 0.0
    upper_limit = 1.0
    best_value = lower_limit
    max_num_rounds = 100

    num_rounds = 1
    while True:
        value = lower_limit + 0.5 * (upper_limit - lower_limit)
        if accuracy(value) >= min_accuracy:
            best_value = value
            lower_limit = value
        else:
            best_value = lower_limit
            upper_limit = value
        print(
            f'The highest max_mse with accuracy ≥ {min_accuracy:g}% after {num_rounds} rounds of '
            f'binary search: {best_value}\r',
            end = '',
        )
        num_rounds += 1
        if num_rounds > max_num_rounds:
            break

In [4]:
binary_search(min_accuracy=100.0)

In [5]:
binary_search(min_accuracy=95.0)